In [4]:
import pandas as pd

# HIST_PRICE_DF = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Prices.xlsx')
HIST_PRICE_DF_RAW = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Prices.csv', sep='|')

#Create a sample for code testing purposes
HIST_PRICE_DF = HIST_PRICE_DF_RAW[HIST_PRICE_DF_RAW['TickName'].isin(['CBA.AX','WES.AX'])]

#create a baselist for adding on the feature engineered variables
TICKER_LIST = HIST_PRICE_DF['TickName'].drop_duplicates().reset_index(drop=True)

In [5]:
# Freature Engineering at daily level
# HIST_PRICE_DF = DF[DF['TickName'].isin(['CBA.AX','WES.AX'])].reset_index(drop=True)

#Movement relative to Open price
HIST_PRICE_DF['HIGHtakeLOW'] = (HIST_PRICE_DF['High']-HIST_PRICE_DF['Low']).abs()
HIST_PRICE_DF['CLOSEtakeOPEN'] = (HIST_PRICE_DF['Close']-HIST_PRICE_DF['Open']).abs()
HIST_PRICE_DF['HIGHdivideLOW'] = HIST_PRICE_DF['High']/HIST_PRICE_DF['Low']-1
HIST_PRICE_DF['CLOSEdivideOPEN'] = HIST_PRICE_DF['Close']/HIST_PRICE_DF['Open']-1

HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] > HIST_PRICE_DF['Open'] , ['V_CloseHigher'] ] = HIST_PRICE_DF['CLOSEdivideOPEN']
HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] < HIST_PRICE_DF['Open'] , ['V_CloseLower'] ] = HIST_PRICE_DF['CLOSEdivideOPEN']

HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] > HIST_PRICE_DF['Open'] , ['C_CloseHigher'] ] = 1
HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] < HIST_PRICE_DF['Open'] , ['C_CloseLower'] ] = 1

# Create flags for aggregation
HIST_PRICE_DF['Freq'] = 1
HIST_PRICE_DF.loc[HIST_PRICE_DF['Close'] > HIST_PRICE_DF['Open'] , ['C_CLOSEaboveOPEN'] ] = 1
HIST_PRICE_DF.loc[HIST_PRICE_DF['Open'] > HIST_PRICE_DF['Low'] , ['C_LOWaboveOpen'] ] = 1




<ipython-input-5-0952973cce55>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HIST_PRICE_DF['HIGHtakeLOW'] = (HIST_PRICE_DF['High']-HIST_PRICE_DF['Low']).abs()
<ipython-input-5-0952973cce55>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HIST_PRICE_DF['CLOSEtakeOPEN'] = (HIST_PRICE_DF['Close']-HIST_PRICE_DF['Open']).abs()
<ipython-input-5-0952973cce55>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [53]:

# Create session level features
HIST_PRICE_DF['TickName_Plus'] = HIST_PRICE_DF['TickName'] + HIST_PRICE_DF['C_CloseHigher'].astype(str)

HIST_PRICE_DF['TickName_Plus'] = HIST_PRICE_DF['TickName'] + HIST_PRICE_DF['C_CloseHigher'].astype(str)
HIST_PRICE_DF['run_num'] = HIST_PRICE_DF['TickName_Plus'].ne(HIST_PRICE_DF['TickName_Plus'].shift()).cumsum()
HIST_PRICE_DF['run_length'] = HIST_PRICE_DF.groupby((HIST_PRICE_DF['TickName_Plus'] != HIST_PRICE_DF['TickName_Plus'].shift(1)).cumsum()).cumcount()+1
# HIST_PRICE_DF['vol'] = HIST_PRICE_DF.groupby(HIST_PRICE_DF['TickName_Plus'] != (HIST_PRICE_DF['TickName_Plus'].shift()).sum()

HIST_PRICE_DF['TickName_PlusSession'] = HIST_PRICE_DF['TickName'] + HIST_PRICE_DF['run_num'].astype(str)


HIST_PRICE_DF['Sess_movement'] = HIST_PRICE_DF.groupby('TickName_PlusSession')['CLOSEdivideOPEN'].cumsum()
HIST_PRICE_DF.loc[ HIST_PRICE_DF['C_CloseHigher']  == 1 , ['Sess_movement_high'] ] = HIST_PRICE_DF['Sess_movement']
HIST_PRICE_DF.loc[ HIST_PRICE_DF['C_CloseLower']  == 1 , ['Sess_movement_low'] ] = HIST_PRICE_DF['Sess_movement'].abs()

# PRICE_MEDIAN = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].median()



In [ ]:
HIST_PRICE_DF.to_excel('/Users/joezhou/Downloads/_temp.xlsx')  


In [57]:
# https://stackoverflow.com/questions/48276812/how-to-calculate-running-total-and-reset-when-value-change-with-python


# HIST_PRICE_DF_Sess = HIST_PRICE_DF.groupby(['TickName', 'TickName_PlusSession'])['run_length','Sess_movement_low','Sess_movement_high'].max()

HIST_PRICE_DF_med = HIST_PRICE_DF.groupby(['TickName'])['HIGHtakeLOW','CLOSEtakeOPEN','HIGHdivideLOW','CLOSEdivideOPEN','V_CloseHigher','V_CloseLower'].median().add_suffix('_med')
HIST_PRICE_DF_Sum = HIST_PRICE_DF.groupby(['TickName'])['Freq','C_CloseHigher','C_CloseLower','C_LOWaboveOpen'].sum().add_suffix('_sum')
HIST_PRICE_DF_max = HIST_PRICE_DF.groupby(['TickName'])['run_length','High','Close'].max().add_suffix('_max')


HIST_PRICE_DF_ADDED = pd.merge(HIST_PRICE_DF_Sum,HIST_PRICE_DF_med,on='TickName',how='left').merge(HIST_PRICE_DF_max,on='TickName',how='left')


# Calculate probability of increase
HIST_PRICE_DF_ADDED['P_CloseHigher'] = HIST_PRICE_DF_ADDED['C_CloseHigher_sum'] / HIST_PRICE_DF_ADDED['Freq_sum'] 
HIST_PRICE_DF_ADDED['Mvmt_Likihood'] = (HIST_PRICE_DF_ADDED['P_CloseHigher']*HIST_PRICE_DF_ADDED['V_CloseHigher_med'])+((1-HIST_PRICE_DF_ADDED['V_CloseLower_med'])*HIST_PRICE_DF_ADDED['V_CloseLower_med'])



<ipython-input-57-a32d472028b2>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_med = HIST_PRICE_DF.groupby(['TickName'])['HIGHtakeLOW','CLOSEtakeOPEN','HIGHdivideLOW','CLOSEdivideOPEN','V_CloseHigher','V_CloseLower'].median().add_suffix('_med')
<ipython-input-57-a32d472028b2>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_Sum = HIST_PRICE_DF.groupby(['TickName'])['Freq','C_CloseHigher','C_CloseLower','C_LOWaboveOpen'].sum().add_suffix('_sum')
<ipython-input-57-a32d472028b2>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_max = HIST_PRICE_DF.groupby(['TickName'])['run_length','High','Close'].max().add_suffix('_max')


In [50]:
#Feature Engineering at Ticker level

# BASELINE = BASELINE.columns = ['TickerName']
PRICE_MEDIAN = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].median()
PRICE_STD_DEV = HIST_PRICE_DF.groupby("TickName")["absmvmtOPEN"].std()


TICKER_LIST_ADDED = pd.merge(TICKER_LIST,PRICE_MEDIAN,on='TickName').rename(columns={'absmvmtOPEN': 'absmvmtOPEN_MEDIAN'})


TICKER_LIST_ADDED = pd.merge(TICKER_LIST_ADDED,PRICE_STD_DEV,on='TickName').rename(columns={'absmvmtOPEN': 'absmvmtOPEN_STD'})



KeyError: 'Column not found: absmvmtOPEN'

In [58]:


HIST_PRICE_DF_ADDED.to_excel('/Users/joezhou/Downloads/ALL_Prices_Features.xlsx')  
